In [ ]:

import cv2
import os
import numpy as np
import tensorflow as tf
from keras import layers
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import time


In [ ]:
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)  # or any {DEBUG, INFO, WARN, ERROR, FATAL}


In [ ]:
batch_size = 8
img_height = 256
img_width = 256
train_data_dir = "tdata"

In [ ]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        #print(f'{filename} \n')
        img = plt.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images


In [ ]:
x = []
y = []
xv = []
yv = []
tdata = 'td4'
FF = 'Flatfeet'
NF = 'Normalfeet'
FFv = 'fft'
NFv = 'nft'
np.random.seed(42)
import random 
x_=  load_images_from_folder(tdata+'/'+FF);
for o in x_:
    y.append(np.array([1]))
x.extend(x_)

x1_=  load_images_from_folder(tdata+'/'+NF);
for o in x1_:
    y.append(np.array([0]))
x.extend(x1_)

xt_=  load_images_from_folder(tdata+'/'+FFv);
for o in xt_: 
    yv.append(np.array([1]))
xv.extend(xt_)

x1t_=  load_images_from_folder(tdata+'/'+NFv);
for o in x1t_:
    yv.append(np.array([0]))
xv.extend(x1t_)
import sklearn

x,y = sklearn.utils.shuffle(x,y)

In [ ]:
for i in range(50,55):
    print(y[i])
    plt.imshow(x[i])
    plt.show()
    time.sleep(1)


In [ ]:
model = Sequential([
  layers.Rescaling(1/255, input_shape=(img_height, img_width, 3)),
  layers.RandomFlip("horizontal",input_shape=(img_height, img_width,3)),
  layers.RandomContrast(0.4),
  layers.RandomRotation(0.4),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(1,activation='sigmoid'),
])
model.compile(optimizer='rmsprop',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

model.summary()

In [ ]:
epochs = 20
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='bots',
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history = model.fit(np.array(x), np.array(y) ,batch_size=batch_size, epochs=epochs, validation_data=(np.array(xv),np.array(yv)),shuffle=True,
callbacks=[
    tf.keras.callbacks.ReduceLROnPlateau()
    ])
model.save('model11k2')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
img = plt.imread('t19.jpeg')

print(np.shape(img))
data_augmentation = Sequential(
  [
     layers.Resizing(256,256)
  ]
)
img = data_augmentation(img)
plt.imshow(img/255)
img=np.expand_dims(img, 0)

res =model.predict(img)
res

In [ ]:
import sklearn
model = tf.keras.models.load_model('model11')

y_pred = model.predict(np.array(xv))
y_pred = np.round(y_pred).tolist()




In [ ]:
from sklearn import metrics
conf_matrix = metrics.confusion_matrix(yv, y_pred)



In [ ]:
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=(conf_matrix[i, j]), va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()